---

# 13.1.1 Hyperparameter optimization

## Using KerasTuner

Conda (recommended)

```python
!conda install -c conda-forge keras-tuner
```

Or pip

```python
!pip install keras-tuner -q
```

In [3]:
import tensorflow as tf
import keras_tuner as kt

**A KerasTuner model-building function**

In [4]:
def build_model(hp):
    units = hp.Int(name="units", min_value=16, max_value=64, step=16)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(units, activation="relu"),
        tf.keras.layers.Dense(10, activation="softmax")
    ])
    optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"])
    return model

**A KerasTuner `HyperModel`**

In [5]:
class SimpleMLP(kt.HyperModel):
    def __init__(self, num_classes):
        self.num_classes = num_classes

    def build(self, hp):
        units = hp.Int(name="units", min_value=16, max_value=64, step=16)
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(units, activation="relu"),
            tf.keras.layers.Dense(self.num_classes, activation="softmax")
        ])
        optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
        model.compile(
            optimizer=optimizer,
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"])
        return model

hypermodel = SimpleMLP(num_classes=10)

In [6]:
tuner = kt.BayesianOptimization(
    build_model,
    objective="val_accuracy",
    max_trials=100,
    executions_per_trial=2,
    directory="mnist_kt_test",
    overwrite=True,
)

In [7]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': None}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}


In [8]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape((-1, 28 * 28)).astype("float32") / 255
x_test = x_test.reshape((-1, 28 * 28)).astype("float32") / 255
x_train_full = x_train[:]
y_train_full = y_train[:]
num_val_samples = 10000
x_train, x_val = x_train[:-num_val_samples], x_train[-num_val_samples:]
y_train, y_val = y_train[:-num_val_samples], y_train[-num_val_samples:]
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5),
]
tuner.search(
    x_train, y_train,
    batch_size=256,
    epochs=100,
    validation_data=(x_val, y_val),
    callbacks=callbacks,
    verbose=2,
)

Trial 100 Complete [00h 00m 30s]
val_accuracy: 0.9733000099658966

Best val_accuracy So Far: 0.976500004529953
Total elapsed time: 00h 55m 09s


**Querying the best hyperparameter configurations**

In [14]:
top_n = 4
best_hps = tuner.get_best_hyperparameters(top_n)

In [10]:
def get_best_epoch(hp):
    model = build_model(hp)
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss", mode="min", patience=10)
    ]
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=100,
        batch_size=128,
        callbacks=callbacks)
    val_loss_per_epoch = history.history["val_loss"]
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
    print(f"Best epoch: {best_epoch}")
    return best_epoch

In [15]:
def get_best_trained_model(hp):
    best_epoch = get_best_epoch(hp)
    model = build_model(hp)
    history = model.fit(
        x_train_full, y_train_full,
        batch_size=128, epochs=int(best_epoch * 1.2)
    )
    return model, history

best_models = []
for hp in best_hps:
    model, history = get_best_trained_model(hp)
    model.evaluate(x_test, y_test)
    best_models.append(model)

Epoch 1/100
391/391 [==============================] - 2s 3ms/step - loss: 0.4319 - accuracy: 0.8833 - val_loss: 0.2283 - val_accuracy: 0.9365
Epoch 2/100
391/391 [==============================] - 1s 3ms/step - loss: 0.2135 - accuracy: 0.9384 - val_loss: 0.1723 - val_accuracy: 0.9532
Epoch 3/100
391/391 [==============================] - 1s 3ms/step - loss: 0.1638 - accuracy: 0.9524 - val_loss: 0.1530 - val_accuracy: 0.9576
Epoch 4/100
391/391 [==============================] - 1s 3ms/step - loss: 0.1330 - accuracy: 0.9610 - val_loss: 0.1279 - val_accuracy: 0.9643
Epoch 5/100
391/391 [==============================] - 1s 3ms/step - loss: 0.1122 - accuracy: 0.9676 - val_loss: 0.1273 - val_accuracy: 0.9631
Epoch 6/100
391/391 [==============================] - 1s 3ms/step - loss: 0.0972 - accuracy: 0.9715 - val_loss: 0.1065 - val_accuracy: 0.9690
Epoch 7/100
391/391 [==============================] - 1s 3ms/step - loss: 0.0855 - accuracy: 0.9750 - val_loss: 0.1040 - val_accuracy: 0.9695

In [ ]:
best_models = tuner.get_best_models(top_n)